# Genotation primers design example

Note: this notebooks if for primer-specific features. For general features use explore_mouse example


In [1]:
import pprint
from pathlib import *
from pycomfort.files import *

In [2]:
import sys

base = Path("..")
local = (base / "genotations").resolve()
if local.exists():
    sys.path.insert(0, Path("..").absolute().as_posix())
    sys.path.insert(0, local)
    print(sys.path)
else:
    base = Path(".")
%load_ext autoreload
%autoreload 2

[PosixPath('/home/antonkulaga/genotations/genotations'), '/home/antonkulaga/genotations/examples/..', '/home/antonkulaga/genotations/examples', '/home/antonkulaga/micromamba/envs/genotations/lib/python310.zip', '/home/antonkulaga/micromamba/envs/genotations/lib/python3.10', '/home/antonkulaga/micromamba/envs/genotations/lib/python3.10/lib-dynload', '', '/home/antonkulaga/micromamba/envs/genotations/lib/python3.10/site-packages']


In [3]:
examples = base / "examples"
data = examples / "data"
tprint(data)

data
	PRJNA543661_transcripts.parquet


In [10]:
import polars as pl
from genotations import genomes
from genotations.genomes import Annotations
from genotations import *
from genotations import ensembl
from genotations.quantification import *
from genotations.genomes import *

In [11]:
pl.Config.set_tbl_width_chars(10000)
pl.Config.set_fmt_str_lengths(1000)
pl.Config.set_tbl_rows(20)

polars.cfg.Config

In [12]:
human = ensembl.human
mouse = ensembl.mouse
mouse

In [158]:
traf_3 = mouse.annotations.with_gene_name_in("Traf3").exons().with_sequences(mouse.genome)
traf_3.annotations_df

seqname,source,feature,start,end,score,strand,frame,attribute,gene,gene_name,transcript_biotype,transcript,transcript_name,exon,exon_number,transcript_exon,coordinates,sequence
str,str,str,u64,u64,str,cat,str,str,str,str,str,str,str,str,u64,str,list[str],str
"""12""","""ensembl_havana""","""exon""",111132804,111133036,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021277""; gene_version ""17""; transcript_id ""ENSMUST00000117269""; transcript_version ""8""; exon_number ""1""; gene_name ""Traf3""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Traf3-203""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS36563""; exon_id ""ENSMUSE00000712457""; exon_version ""2""; tag ""basic""; transcript_support_level ""1 (assigned to previous version 7)"";""","""ENSMUSG00000021277""","""Traf3""","""protein_coding""","""ENSMUST00000117269""","""Traf3-203""","""ENSMUSE00000712457""",1,"""Traf3-203_1""","[""12"", ""+"", ... ""111133036""]","""agcgcgcgAGAGGAAGTGCCAGCGCGAGGGTGCGTGAGGCGGCGCGGCCGGCGGCCGCCGCGTGCGCGAGCCGGGTTGCAGCCCAGCAGGGACTTTCCAGCcggcggcggcggcggcggcCGCCGGCCCTTCCCCGCCCCCCGACATGGGGCTGCCCGGGGAGCTGGACGCTGCAGAAGGCGGCGGAGGATGCGCGCGGCGCCCCTGAgccggccgaacgggcgggcctcggg"""
"""12""","""ensembl_havana""","""exon""",111203996,111204254,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021277""; gene_version ""17""; transcript_id ""ENSMUST00000117269""; transcript_version ""8""; exon_number ""2""; gene_name ""Traf3""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Traf3-203""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS36563""; exon_id ""ENSMUSE00000311868""; exon_version ""2""; tag ""basic""; transcript_support_level ""1 (assigned to previous version 7)"";""","""ENSMUSG00000021277""","""Traf3""","""protein_coding""","""ENSMUST00000117269""","""Traf3-203""","""ENSMUSE00000311868""",2,"""Traf3-203_2""","[""12"", ""+"", ... ""111204254""]","""AATTTCAGTTTCCTAAGATGGAGTCAAGCAAAAAGATGGATGCTGCTGGCACACTGCAGCCTAACCCACCCCTAAAGCTGCAGCCTGATCGCGGCGCAGGGTCCGTGCTCGTGCCGGAGCAAGGAGGCTACAAGGAGAAGTTTGTGAAGACGGTGGAAGACAAGTACAAGTGCGAGAAGTGCCGCCTGGTGCTGTGCAACCCGAAGCAGACGGAGTGTGGCCACCGGTTCTGCGAGAGCTGCATGGCCGCCCTGCTGAG"""
"""12""","""ensembl_havana""","""exon""",111205463,111205514,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021277""; gene_version ""17""; transcript_id ""ENSMUST00000117269""; transcript_version ""8""; exon_number ""3""; gene_name ""Traf3""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Traf3-203""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS36563""; exon_id ""ENSMUSE00000116300""; exon_version ""2""; tag ""basic""; transcript_support_level ""1 (assigned to previous version 7)"";""","""ENSMUSG00000021277""","""Traf3""","""protein_coding""","""ENSMUST00000117269""","""Traf3-203""","""ENSMUSE00000116300""",3,"""Traf3-203_3""","[""12"", ""+"", ... ""111205514""]","""CTCCTCCAGTCCAAAATGCACAGCGTGCCAAGAAAGCATCATCAAAGACAAG"""
"""12""","""ensembl_havana""","""exon""",111208877,111208981,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021277""; gene_version ""17""; transcript_id ""ENSMUST00000117269""; transcript_version ""8""; exon_number ""4""; gene_name ""Traf3""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Traf3-203""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS36563""; exon_id ""ENSMUSE00000116306""; exon_version ""2""; tag ""basic""; transcript_support_level ""1 (assigned to previous version 7)"";""","""ENSMUSG00000021277""","""Traf3""","""protein_coding""","""ENSMUST00000117269""","""Traf3-203""","""ENSMUSE00000116306""",4,"""Traf3-203_4""","[""12"", ""+"", ... ""111208981""]","""GTGTTTAAGGATAATTGCTGCAAGAGAGAGATTCTGGCCCTTCAGGTCTACTGTCGGAATGAAGGCAGAGGTTGTGCGGAGCAGCTGAC

In [177]:
 mouse.annotations.with_gene_name_in("Traf3").genes().annotations_df.select(['start', 'end'])

start,end
u64,u64
111132804,111233587


In [163]:
proper_traf_3 = traf_3.with_transcript_name_in("Traf3-201", "Traf3-204")
proper_traf_3.annotations_df

seqname,source,feature,start,end,score,strand,frame,attribute,gene,gene_name,transcript_biotype,transcript,transcript_name,exon,exon_number,transcript_exon,coordinates,sequence
str,str,str,u64,u64,str,cat,str,str,str,str,str,str,str,str,u64,str,list[str],str
"""12""","""ensembl_havana""","""exon""",111132897,111133036,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021277""; gene_version ""17""; transcript_id ""ENSMUST00000021706""; transcript_version ""11""; exon_number ""1""; gene_name ""Traf3""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Traf3-201""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS26175""; exon_id ""ENSMUSE00000779387""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 10)"";""","""ENSMUSG00000021277""","""Traf3""","""protein_coding""","""ENSMUST00000021706""","""Traf3-201""","""ENSMUSE00000779387""",1,"""Traf3-201_1""","[""12"", ""+"", ... ""111133036""]","""TTTCCAGCcggcggcggcggcggcggcCGCCGGCCCTTCCCCGCCCCCCGACATGGGGCTGCCCGGGGAGCTGGACGCTGCAGAAGGCGGCGGAGGATGCGCGCGGCGCCCCTGAgccggccgaacgggcgggcctcggg"""
"""12""","""ensembl_havana""","""exon""",111186245,111186355,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021277""; gene_version ""17""; transcript_id ""ENSMUST00000021706""; transcript_version ""11""; exon_number ""2""; gene_name ""Traf3""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Traf3-201""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS26175""; exon_id ""ENSMUSE00000436710""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 10)"";""","""ENSMUSG00000021277""","""Traf3""","""protein_coding""","""ENSMUST00000021706""","""Traf3-201""","""ENSMUSE00000436710""",2,"""Traf3-201_2""","[""12"", ""+"", ... ""111186355""]","""GTACAGGGTCCCCATTACTTGAAGGATAAGGCTGGCACGGCTCCGACGTCTGTGTGGAAGCTTCTCCCTCCCTTCTGAGCTTCTCTAGACTCCTTACAGCGCACGGCACAG"""
"""12""","""ensembl_havana""","""exon""",111203996,111204254,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021277""; gene_version ""17""; transcript_id ""ENSMUST00000021706""; transcript_version ""11""; exon_number ""3""; gene_name ""Traf3""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Traf3-201""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS26175""; exon_id ""ENSMUSE00000311868""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 10)"";""","""ENSMUSG00000021277""","""Traf3""","""protein_coding""","""ENSMUST00000021706""","""Traf3-201""","""ENSMUSE00000311868""",3,"""Traf3-201_3""","[""12"", ""+"", ... ""111204254""]","""AATTTCAGTTTCCTAAGATGGAGTCAAGCAAAAAGATGGATGCTGCTGGCACACTGCAGCCTAACCCACCCCTAAAGCTGCAGCCTGATCGCGGCGCAGGGTCCGTGCTCGTGCCGGAGCAAGGAGGCTACAAGGAGAAGTTTGTGAAGACGGTGGAAGACAAGTACAAGTGCGAGAAGTGCCGCCTGGTGCTGTGCAACCCGAAGCAGACGGAGTGTGGCCACCGGTTCTGCGAGAGCTGCATGGCCGCCCTGCTGAG"""
"""12""","""ensembl_havana""","""exon""",111205463,111205514,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021277""; gene_version ""17""; transcript_id ""ENSMUST00000021706""; transcript_version ""11""; exon_number ""4""; gene_name ""Traf3""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Traf3-201""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS26175""; exon_id ""ENSMUSE00000116300""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 10)"";""","""ENSMUSG00000021277""","""Traf3""","""protein_coding""","""ENSMUST00000021706""","""Traf3-201""","""ENSMUSE00000116300""",4,"""Traf3-201_4""","[""12"", ""+"", ... ""111205514""]","""CTCCTCC

In [172]:
from genotations.intersections import *
intervals_traf_3 = TranscriptIntersection.extract_intervals(proper_traf_3)
intervals_traf_3

[TranscriptIntersection(transcripts={'Traf3-201_1'}, contig='12', start=111132897, end=111133036), TranscriptIntersection(transcripts={'Traf3-204_1'}, contig='12', start=111132920, end=111133036), TranscriptIntersection(transcripts={'Traf3-204_2'}, contig='12', start=111133433, end=111133541), TranscriptIntersection(transcripts={'Traf3-201_2'}, contig='12', start=111186245, end=111186355), TranscriptIntersection(transcripts={'Traf3-204_3'}, contig='12', start=111186245, end=111186355), TranscriptIntersection(transcripts={'Traf3-201_3'}, contig='12', start=111203996, end=111204254), TranscriptIntersection(transcripts={'Traf3-204_4'}, contig='12', start=111203996, end=111204254), TranscriptIntersection(transcripts={'Traf3-201_4'}, contig='12', start=111205463, end=111205514), TranscriptIntersection(transcripts={'Traf3-204_5'}, contig='12', start=111205463, end=111205511), TranscriptIntersection(transcripts={'Traf3-201_5'}, contig='12', start=111208877, end=111208981), TranscriptIntersect

In [ ]:
for i1 in intervals_traf_3:
    for i2 in intervals_traf_3:
        if i1 != i2:
            from genotations.primers import *
            res = suggest_interval_primers(mouse.genome.get_seq("12", 111132804, 111233587), i1, i2)
            print(res)

In [152]:
from genotations.primers import suggest_primers_by_exons, check_primer_results
import pprint
test = suggest_primers_by_exons(traf_3.get_transcript_sequences(), 2, 3)
test

OrderedDict([('Traf3-201',
              <genotations.primers.PrimerResults at 0x7f68d50f03a0>),
             ('Traf3-202',
              <genotations.primers.PrimerResults at 0x7f68d50f1330>),
             ('Traf3-203',
              <genotations.primers.PrimerResults at 0x7f68d50f2ec0>),
             ('Traf3-204',
              <genotations.primers.PrimerResults at 0x7f68d50f0640>),
             ('Traf3-205',
              <genotations.primers.PrimerResults at 0x7f68d50f0c10>)])

In [153]:
check_primer_results(test["Traf3-201"], test["Traf3-204"])

In [156]:
len(test["Traf3-201"].get_cleaned_results())

5

In [77]:
def make_primer(anno: Annotations, exon_1: int, exon_2: int, genome: genomepy.genome = None):
    df = anno.get_transcript_sequences(transcript, genome)
    return df.apply(lambda r: seq(r).reduce(lambda a,b : a+b))

In [76]:
make_primer(traf_3)

shape: (5, 4)
┌─────────────────┬────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ transcript_name ┆ transcript_exon                                    ┆ sequence                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    ┆ mRNA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [14]:
from genotations.primers import suggest_primers
from genotations.intersections import *

In [16]:
TranscriptIntersection.extract_intervals(traf3)

NotFoundError: coordinates